# Recommendations Data Mart Builder

The purpose of this code is to build the final datamart for the boxcoxrox project based on the csv files from the sentiment, ranking and LDA modules.  


In [1]:
import pandas as pd

def shorten(x):
    if type(x) != str:
        x = ""
    if len(x) > 100:
        x = "{}...".format(x[:100])
    return x


#Specify the paths to the individual files here...
product_file = "pet_products_all.csv"
lda_file = "LDA_Sklearn/LDA_Category_Topic.csv"
sentiment_file = "sentiment/pets_sentiment_grouped.csv"
ranking_file = "quantiles/ranking_output.csv"
validation_file = "link_validator/valid_links.csv"
blocklist_file = "blocklist.csv"

print("Blocklist.")
blocklist = pd.read_csv(blocklist_file, index_col=False)
print("Asins in blocklist: {}".format(len(blocklist)))

print("Product")
product_df = pd.read_csv(product_file)
print("Products before de-duplication: {}".format(len(product_df)))
product_df.drop_duplicates(subset='asin', keep="first", inplace=True)
print("Products after de-duplication: {}".format(len(product_df)))

product_df['description'] = product_df.apply(lambda x: shorten(x['description']), axis=1)
product_df['amazon_category'] = product_df['category']
product_df.drop('category', inplace=True, axis=1)
print(product_df.columns)

print("\nLDA")
lda_df = pd.read_csv(lda_file, header=0)
print(lda_df.columns)

print("\nSentiment")
sentiment_df = pd.read_csv(sentiment_file, index_col=0)
sentiment_df = sentiment_df.rename({'mean':'mean_sentiment', 'median':'median_sentiment', 'max':'max_sentiment', 'std':'std_sentiment', 'min':'min__sentiment'}, axis=1)
print(sentiment_df.columns)

print("\nRanking")
ranking_df = pd.read_csv(ranking_file, index_col=0)
ranking_df = ranking_df.rename({'overall_average':'overall_average_ranking', 'overall_count':'overall_count_ranking', 'overall_stdev':'overall_stdev_ranking',
       'average_length_of_review_text':'ave_length_review_text', 'percentile':'percentile_ranking'}, axis=1)
print(ranking_df.columns)

print("\nValidation")
validation_df = pd.read_csv(validation_file)
print(validation_df.columns)



Blocklist.
Asins in blocklist: 16
Product
Products before de-duplication: 205999
Products after de-duplication: 198265
Index(['asin', 'title', 'description', 'amazon_category'], dtype='object')

LDA
Index(['asin', 'category', 'topic_name'], dtype='object')

Sentiment
Index(['asin', 'std_sentiment', 'median_sentiment', 'max_sentiment',
       'mean_sentiment', 'min__sentiment'],
      dtype='object')

Ranking
Index(['asin', 'overall_average_ranking', 'overall_count_ranking',
       'overall_stdev_ranking', 'ave_length_review_text', 'rank',
       'percentile_ranking'],
      dtype='object')

Validation
Index(['asin', 'valid_link'], dtype='object')


The following will join all the data frames and save the results as a single csv...

In [2]:
print(product_df.columns)
print(lda_df.columns)

def check_asin(asin, df):
    x=df.copy()
    x=x[x['asin']==asin]
    if len(x)>0:
        return True
    else:
        return False

# Merging LDA data onto products...
all_df = pd.merge(product_df, lda_df, on="asin", how="inner")

print("Records after merging topics_df: {}".format(all_df.shape[0]))
all_df = pd.merge(all_df, sentiment_df, suffixes=("", "_sentiment"), on="asin", how="inner")

print(all_df.shape)

all_df = pd.merge(all_df, ranking_df, on="asin", suffixes=("", "_ranking"), how="inner")
all_df = all_df.drop_duplicates()

def get_val(x):
    if (x==1 or x==0):
        return x
    else:
        return 0

all_df = all_df.merge(validation_df, on='asin', how='left')
all_df['valid_link'] = all_df.apply(lambda row: get_val(row['valid_link']), axis=1)

print(all_df.shape)
all_df.to_csv("all_products_4.11.csv", index=False)
print(all_df.describe())
                  

Index(['asin', 'title', 'description', 'amazon_category'], dtype='object')
Index(['asin', 'category', 'topic_name'], dtype='object')
Records after merging topics_df: 198260
(198260, 11)
(199993, 18)
       std_sentiment  median_sentiment  max_sentiment  mean_sentiment  \
count  137371.000000     199993.000000  199993.000000   199993.000000   
mean        0.400319          0.538444       0.748337        0.483135   
std         0.207737          0.357458       0.357321        0.337751   
min         0.000000         -0.994100      -0.994100       -0.994100   
25%         0.257037          0.410000       0.670500        0.329967   
50%         0.410079          0.635350       0.899300        0.537425   
75%         0.520523          0.789900       0.967400        0.716000   
max         1.397879          0.999900       1.000000        0.999900   

       min__sentiment  overall_average_ranking  overall_count_ranking  \
count   199993.000000            199993.000000          199993.000000 

In [3]:
test_df = pd.read_csv("all_products_4.11.csv")
test_df

asin                                              title  \
0       061539972X  DR.REXY Hemp Oil for Dogs and Cats - 100% Orga...   
1       0615553605             Pet Qwerks Treat Cookbook with Cutters   
2       0760339597                         The Best of 101 Dog Tricks   
3       0793816793  Lionfishes and Other Scorpionfishes: The Compl...   
4       0972585419  Pet Media Feathered Phonics The Easy Way To Te...   
...            ...                                                ...   
199989  B01HJ9ULQW  Sunlilee 3Pcs Pack Adjustable Pet Dog Puppy Ca...   
199990  B01HJ9OZZA  Lesypet Pet Carrier for Small Dogs, Cat/Mesh B...   
199991  B01HJABKBQ  HAOCOO Dog Life Jacket Vest Saver Safety Swims...   
199992  B01HJCJ8KO  TEEFAN Retractable Leash 10ft for Pets Puppy D...   
199993  B01HJDIJQ2                                Fairwin Pet Product   

                                              description  \
0       Dr. Rexy hemp oil has powerful anti-inflammato...   
1       Know exactly what your dog is eating with 101 ...   
2                                                     ,,,   
3       ,The venomous but beautiful scorpionfishes are...   
4       Volume 1: 96 Words &amp; Phrases! This is the ...   
...                                                   ...   
199989  <br>100% brand new and high quality. <br>A bes...   
199990  <br><b>Color:Coffee,Rose</b> <br><b>Size:</b> ...   
199991  Are you still worry about the safety issue abo...   
199992  Are you looking for a sturdy leash for your lo...   
199993  <b>100% Real Leather Dog Lead</b> - We use onl...   

                                          amazon_category category topic_name  \
0       Pet Supplies,Dogs,Health Supplies,Hip &amp; Jo...      dog      dog_0   
1                                       Pet Supplies,Dogs      dog      dog_2   
2                              Pet Supplies,Dogs,Food,Dry      dog      dog_6   
3                                                     NaN     fish     fish_1   
4              Pet Supplies,Top Selection from AmazonPets     bird     bird_3   
...                                                   ...      ...        ...   
199989  Pet Supplies,Dogs,Collars, Harnesses & Leashes...    other    other_0   
199990  Pet Supplies,Dogs,Carriers & Travel Products,B...      cat      cat_0   
199991  Pet Supplies,Dogs,Apparel & Accessories,Lifeja...      dog      dog_0   
199992  Pet Supplies,Dogs,Collars, Harnesses & Leashes...      dog      dog_0   
199993  Pet Supplies,Dogs,Collars, Harnesses & Leashes...      dog      dog_0   

        std_sentiment  median_sentiment  max_sentiment  mean_sentiment  \
0            0.401246           0.76270         0.8573        0.595675   
1            0.472375           0.76500         0.9730        0.625800   
2            0.273193           0.68420         0.9917        0.602211   
3            0.204703           0.87900         0.9460        0.790817   
4            0.541617           0.38750         0.9730        0.178232   
...               ...               ...            ...             ...   
199989            NaN           0.87390         0.8739        0.873900   
199990            NaN           0.87880         0.8788        0.878800   
199991       0.306682           0.67325         0.9733        0.664659   
199992       0.213721           0.71060         0.8665        0.662225   
199993       0.298110           0.62490         0.9724        0.609867   

        min__sentiment  overall_average_ranking  overall_count_ranking  \
0               0.0000                 5.000000                    4.0   
1              -0.5765                 4.111111                    9.0   
2               0.1436                 4.222222                    9.0   
3               0.4389                 4.500000                    6.0   
4              -0.9311                 3.015873                   63.0   
...                ...                      ...                    ...   
199989          0.8739  

Prepare data for visualization.

The CSV for visualization needs the following columns:

,asin,title,description,category,topic_name,topic_rank,bubble_color,bubble_size,clean_link

| pipeline field | products_prepped |
|----------------|------------------|
| asin           | asin             |
| title          | title            |
| description    | description      |
| category       | category         |
| topic_name     | topic_name       |
| topic_rank     | huh.             |
| bubble_color   | mean_sentiment   |
| bubble_size    | overall_count_ranking |
| clean_link     | valid_link       |


In [4]:
#pdf = pd.read_csv('prepped_products_sample.csv')
#print(pdf.describe())

In [5]:
print(all_df['topic_name'].head())

def get_topic_rank_df(topic, df):
    a = df.copy()
    a = a[a['topic_name']==topic]
    a.sort_values('rank', inplace=True)
    a.reset_index(inplace=True)
    a['topic_rank'] = a.index
    a = a[['asin', 'topic_rank']]
    return a

product_prepped = all_df[['asin','title','description','category','topic_name','mean_sentiment','overall_count_ranking', 'valid_link']]
product_prepped.columns = ['asin','title','description','category','topic_name','bubble_color','bubble_size','clean_link']#

temp_dfs = []

topics = list(set(all_df['topic_name']))
for topic in topics:
    topic_df = get_topic_rank_df(topic, all_df)
    temp_dfs.append(product_prepped.merge(topic_df, on="asin", how="inner"))
    
product_prepped = pd.concat(temp_dfs)    
print("Done.")

product_prepped = product_prepped.drop_duplicates()
# Remove bad products.
print("Product count before blocklist: {}".format(len(product_df)))
product_prepped = product_prepped[~product_prepped['asin'].isin(blocklist['asin'])]
product_prepped.drop_duplicates('asin', keep="first", inplace=True)
print("Product count after blocklist:  {}".format(len(product_df)))

#Trim to top 100 products in LDA topic.
product_prepped = product_prepped[product_prepped['topic_rank']<=100]
product_prepped.sort_values(['topic_rank'], inplace=True)
product_prepped.to_csv('products_prepped.csv')
print("Done.")



0     dog_0
1     dog_2
2     dog_6
3    fish_1
4    bird_3
Name: topic_name, dtype: object
Done.
Product count before blocklist: 198265
Product count after blocklist:  198265
Done.


In [6]:
kim_df = pd.read_csv("prepped_products_sample.csv")
display(kim_df.describe())

pipeline_df = pd.read_csv("products_prepped.csv")
display(pipeline_df.describe())

Unnamed: 0   topic_rank  bubble_color  bubble_size   clean_link
count    4238.000000  4238.000000   4238.000000  4238.000000  4238.000000
mean    33866.938414    47.948561      0.599477    38.759556     0.913639
std     46730.743018    29.139580      0.206371   174.789365     0.280930
min         3.000000     1.000000     -0.740050     2.000000     0.000000
25%      3862.750000    22.000000      0.484742     3.000000     1.000000
50%     11942.500000    47.000000      0.615443     7.000000     1.000000
75%     43491.750000    73.000000      0.744316    15.000000     1.000000
max    199953.000000   100.000000      0.979950  2936.000000     1.000000

Unnamed: 0  bubble_color  bubble_size   clean_link   topic_rank
count   3029.000000   3029.000000  3029.000000  3029.000000  3029.000000
mean    2485.027732      0.600188    33.748762     0.867943    47.055464
std     7387.236134      0.244352   155.339097     0.338608    29.440078
min        0.000000     -0.905400     1.000000     0.000000     0.000000
25%       68.000000      0.482457     2.000000     1.000000    21.000000
50%      237.000000      0.624747     6.000000     1.000000    45.000000
75%     1056.000000      0.766100    13.000000     1.000000    72.000000
max    61744.000000      0.996300  2936.000000     1.000000   100.000000